In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TEST_DIR = '../test/'

    # use this for full dataset
    train_images_original = [TRAIN_ORIGINAL_DIR + i for i in os.listdir(TRAIN_ORIGINAL_DIR)]
    train_images_sub = [TRAIN_SUB_DIR + i for i in os.listdir(TRAIN_SUB_DIR)]

    train_images = train_images_original + train_images_sub

    #print (train_images)
    train_images.sort(key=natural_keys)
    print (len(train_images))

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))

    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    print (trainImage.shape)
    trainImage = trainImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    
    return trainImage, trainLabels

In [3]:
trainImage,trainLabels = load_flower_data()

nb_classes = 2

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
np.save('./trainImage.npy', trainImage)
np.save('./trainLabels.npy', trainLabels)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
2002
(2002, 128, 128, 3)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
[INFO] trainImage matrix: 384.38MB
[INFO] trainLabels matrix: 0.0313MB


In [4]:
def leaky_relu( X, leak=0.2):
    f1 = 0.5 * (1 + leak)
    f2 = 0.5 * (1 - leak)
    return f1 * X + f2 * tf.abs(X)   
    

In [5]:
class cDCGAN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, result_dir, log_dir, 
                 test_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.result_dir = result_dir
        self.log_dir = log_dir
        self.test_dir = test_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        self.model_name = "cDCGAN"     # name for checkpoint
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.output_height = 128
        self.output_width = 128

        self.z_dim = 100         # dimension of noise-vector
        self.c_dim = 3  # color dimension
        self.nb_class = 2

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay( 0.0002,
                                             global_step=self.global_step,
                                             decay_steps=20,
                                             decay_rate=0.9,
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # test
        #visual results
        self.onehot = np.eye(self.nb_class)
        self.sample_num = 100  # number of generated images to be saved
        self.visual_team = self.batch_size // self.nb_class  # number of generated images to be saved

        #load_flower_data
        self.data_x = np.load('./trainImage.npy')
        self.data_y= np.load('./trainLabels.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['D_losses'] = []
        self.train_hist['G_losses'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches = len(self.data_x) // self.batch_size

    def discriminator(self, x, y_fill, is_training=True, reuse=False):
        with tf.variable_scope("discriminator", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            b = tf.constant_initializer(0.0)
        
            print("D:x",x.get_shape()) # 32, 32, 3 = 3072
            print("D:y_fill",y_fill.get_shape())
            # concat layer
            
            #卷积核为4*4 输出维度为32
            net1_1 = tf.layers.conv2d(x, 32, [4, 4], strides=(2, 2), padding='same', 
                                     kernel_initializer=W, bias_initializer=b)
            net1_1 = leaky_relu(net1_1, 0.2)
            
            print("D:",net1_1.get_shape())
            
            net1_2 = tf.layers.conv2d(y_fill, 32, [4, 4], strides=(2, 2), padding='same', 
                         kernel_initializer=W, bias_initializer=b)
            net1_2 = leaky_relu(net1_2, 0.2)
            print("D:",net1_2.get_shape())
    
            #把数据和标签进行连接
            net = tf.concat([net1_1, net1_2], 3)
            print("D:",net.get_shape())
            
            #卷积核为4*4 输出维度为128        
            net = tf.layers.conv2d(net, 64, [4, 4], strides=(2, 2), padding='same', 
                                     kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("D:",net.get_shape())
    
            #卷积核为4*4 输出维度为128 
            net = tf.layers.conv2d(net, 128, [4, 4], strides=(2, 2), padding='same', 
                                 kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("D:",net.get_shape())
            
            #卷积核为4*4 输出维度为256
            net = tf.layers.conv2d(net, 256, [4, 4], strides=(2, 2), padding='same', 
                     kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("D:",net.get_shape())
            
            #卷积核为4*4 输出维度为512
            net = tf.layers.conv2d(net, 512, [4, 4], strides=(2, 2), padding='same', 
                 kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("D:",net.get_shape())
            
            #output
            out_logit = tf.layers.conv2d(net, 1, [4, 4], strides=(1, 1), padding='valid', 
                 kernel_initializer=W, bias_initializer=b)
            print("D:out_logit",out_logit.get_shape())
            
            out = tf.nn.sigmoid(out_logit)
            print("D:out",out.get_shape())
            print("------------------------")    

            return out, out_logit, net

    def generator(self, z, y_label, is_training=True, reuse=False):
        with tf.variable_scope("generator", reuse=reuse):
            
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            b = tf.constant_initializer(0.0)
            
            print("G:z",z.get_shape())
            print("G:y_label",y_label.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度256
            net1_1 = tf.layers.conv2d_transpose(z, 256, [4, 4], strides=(1, 1), padding='valid',
                                               kernel_initializer=W, bias_initializer=b)
            net1_1 = leaky_relu(net1_1, 0.2)
            print("G:",net1_1.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度256
            net1_2 = tf.layers.conv2d_transpose(y_label, 256, [4, 4], strides=(1, 1), padding='valid',
                                               kernel_initializer=W, bias_initializer=b)
            net1_2 = leaky_relu(net1_2, 0.2)
            print("G:",net1_2.get_shape())
            
            #把数据和标签进行连接
            # concat layer
            net = tf.concat([net1_1, net1_2], 3)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度512
            net = tf.layers.conv2d_transpose(net, 512, [4, 4], strides=(2, 2), padding='same',
                                             kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度128
            net = tf.layers.conv2d_transpose(net, 128, [4, 4], strides=(2, 2), padding='same',
                                             kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度64
            net = tf.layers.conv2d_transpose(net, 64, [4, 4], strides=(2, 2), padding='same',
                                             kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度32
            net = tf.layers.conv2d_transpose(net, 32, [4, 4], strides=(2, 2), padding='same',
                                             kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度3
            net = tf.layers.conv2d_transpose(net, 3, [4, 4], strides=(2, 2), padding='same',
                                 kernel_initializer=W, bias_initializer=b)
            out = tf.nn.tanh(net)
            print("G:",out.get_shape())
            print("------------------------")

            return out

    def build_model(self):
        #parameters
        image_dims = [self.input_height, self.input_width, self.c_dim]

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=(self.batch_size,self.input_height, self.input_width, self.c_dim), 
                                name='real_images')

        # noises
        self.z = tf.placeholder(tf.float32, shape=(self.batch_size, 1, 1, self.z_dim), name='z')
        
        self.y_label = tf.placeholder(tf.float32, shape=(self.batch_size, 1, 1, self.nb_class), name='y_label')
        self.y_fill = tf.placeholder(tf.float32, shape=(self.batch_size, self.output_width, 
                                                        self.output_height, self.nb_class), name='y_fill')
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of D for real images
        D_real, D_real_logits, _ = self.discriminator(self.x, self.y_fill, is_training=True, reuse=False)
            
        #networks : generator
        G = self.generator(self.z, self.y_label, is_training=True, reuse=False)

        # output of D for fake images
        D_fake, D_fake_logits, _ = self.discriminator(G, self.y_fill, is_training=True, reuse=True)

        # get loss for discriminator
        self.d_loss_real = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
        self.d_loss_fake = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))

        self.d_loss = self.d_loss_real + self.d_loss_fake
        
        # get loss for generator
        self.g_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))
        
        """ Training """
        # divide trainable variables into a group for D and a group for G
        tf_vars = tf.trainable_variables()
        d_vars = [var for var in tf_vars if var.name.startswith('discriminator')]
        g_vars = [var for var in tf_vars if var.name.startswith('generator')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.d_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.d_loss, var_list=d_vars)
            self.g_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.g_loss, var_list=g_vars)
        
        """" Testing """
        # for test
        self.fake_images = self.generator(self.z, self.y_label, is_training=False, reuse=True)

        """ Summary """
        d_loss_real_sum = tf.summary.scalar("d_loss_real", self.d_loss_real)
        d_loss_fake_sum = tf.summary.scalar("d_loss_fake", self.d_loss_fake)
        d_loss_sum = tf.summary.scalar("d_loss", self.d_loss)
        g_loss_sum = tf.summary.scalar("g_loss", self.g_loss)

        # final summary operations
        self.g_sum = tf.summary.merge([d_loss_fake_sum, g_loss_sum])
        self.d_sum = tf.summary.merge([d_loss_real_sum, d_loss_sum])

    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # graph inputs for visualize training results
        self.sample_z = np.random.normal(0, 1, (self.batch_size, 1, 1, 100)).astype(np.float32)

        self.test_images = self.data_x[0:self.batch_size]

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep= self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + 'train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            G_losses = []
            D_losses = []
    
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.data_x.shape[0]), self.data_x.shape[0])
            shuffled_set = self.data_x[shuffle_idxs]
            shuffled_label = self.data_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y_label = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, 1, 1, self.nb_class])
                batch_y_fill = batch_y_label * np.ones([self.batch_size, self.output_height, self.output_width, self.nb_class])
                batch_z = np.random.normal(0, 1, (self.batch_size, 1, 1, 100)).astype(np.float32)
                

                # update D network
                _, summary_str, d_loss = self.sess.run([self.d_optim, self.d_sum, self.d_loss],
                                               feed_dict={self.x: batch_x,
                                                          self.z: batch_z,
                                                          self.y_fill: batch_y_fill,
                                                          self.y_label: batch_y_label}
                                                      )
                self.writer.add_summary(summary_str, counter)

                # update G network
                batch_y = np.random.randint(0, self.nb_class, (self.batch_size, 1)) # <=  <
                batch_y_label = self.onehot[batch_y.astype(np.int32)].reshape([self.batch_size, 1, 1, self.nb_class])
                batch_y_fill  = batch_y_label * np.ones([self.batch_size, self.input_height, self.input_width, self.nb_class])

                # update G once
                _, summary_str, g_loss = self.sess.run([self.g_optim, self.g_sum, self.g_loss],
                                                          feed_dict={self.x: batch_x,
                                                                     self.z: batch_z, 
                                                                     self.y_fill: batch_y_fill,
                                                                     self.y_label: batch_y_label})
                self.writer.add_summary(summary_str, counter)
                
                D_losses.append(d_loss)
                G_losses.append(g_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f" % (epoch_loop, idx, self.num_batches, time.time() - start_time, d_loss, g_loss))

            # After an epoch
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss_d: %.8f, loss_g: %.8f, lf:  %.8f' % (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                        np.mean(D_losses), np.mean(G_losses), rate))
            self.train_hist['D_losses'].append(np.mean(D_losses))
            self.train_hist['G_losses'].append(np.mean(G_losses))
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)

            # show temporal results
            self.visualize_epoch_results(epoch_loop)
            
            # save trainhist for the initial train
            f = open(self.trainhist_dir + '/' + 'train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' + 'train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
    def visualize_img_to_gif(self, path, size, fps = 5):
        print(" [*] Generation Animation GIF start!")
        path_images = [path + '/'+ i for i in os.listdir(path)]
        path_images.sort(key=natural_keys)
        print(" [*] Reading " + str(len(path_images)) + " " +  str(size) + " images start!")
        images = []
        for p in tqdm(path_images):
            img = imageio.imread(p)
            img = cv2.resize(img, size)
            images.append(img)
        print(" [*] Reading " + str(len(path_images)) + " " +  str(size) + " images finished!")
        print(" [*] Generation Animation "+ str(size) + " GIF start! FPS=", fps)
        path_gif = path + '/' + self.model_name + '_epoch%03d' % self.epoch + '_' + self.dataset_name + 'generation_animation.gif'
        imageio.mimsave(path_gif, images, fps = fps)
        print(" [*] Generation Animation "+ str(size) + " GIF finished! FPS=", fps)

    def visualize_test_image(self, epoch, num, times):
        tot_num_samples = num #  100
        path_model = self.checkpoint_dir + '/' + self.model_dir + '/' + self.model_name + '/'+ self.model_name +'.model-'+ str(epoch)
        
        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)
            
        """ random condition, random noise """
        for j in range(times):
            noise = np.random.normal(0, 1, (self.visual_team, 1, 1, self.z_dim))
            fixed_noise = noise 
            fixed_label = np.zeros((self.visual_team , 1))
            #用于显示25组图像
            for i in range(self.nb_class - 1):
                fixed_noise = np.concatenate([fixed_noise, noise], 0)
                temp_label = np.ones((self.visual_team, 1)) + i
                fixed_label = np.concatenate([fixed_label, temp_label], 0)
            fixed_label = self.onehot[fixed_label.astype(np.int32)].reshape((self.batch_size, 1, 1, self.nb_class))
            samples = self.sess.run(self.fake_images, feed_dict={self.z: fixed_noise, self.y_label: fixed_label})
            self.save_cv2_img(samples[:tot_num_samples, :, :, :], tot_num_samples, epoch , j) 
        print(" [*] Finished generating Epoch:", epoch, ",",times*num,"new images!")
        
                                    
    def save_cv2_img(self, images, num, epoch, times):
    # revice image data // M*N*3 // RGB float32 : value must set between 0. with 1.
        for idx in range(num):
            classname = self.classname[idx // self.visual_team]
            img_save = images[idx:idx+1, :, :, :][0, ...]*255
            path = self.test_dir + '/' + classname + '_epoch%03d' % epoch + '_t%02d' % times+ '_%03d' % idx + '.png'
            cv2.imwrite( path, img_save)
              
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True)
        
    def save_matplot_img(self, images, size, image_path):
        # revice image data // M*N*3 // RGB float32 : value must set between 0. with 1.
        for idx in range(100):
            vMin = np.amin(images[idx])
            vMax = np.amax(images[idx])
            img_arr = images[idx].reshape(128*128*3,1) # flatten
            for i, v in enumerate(img_arr):
                img_arr[i] = (v-vMin)/(vMax-vMin)
            img_arr = img_arr.reshape(128,128,3) # M*N*3

            # matplot display
            plt.subplot(10,10,idx+1),plt.imshow(cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB), interpolation='nearest')
            #plt.title("pred.:{}".format(np.argmax(self.data_y[0]),fontsize=10))
            plt.axis("off")

        plt.savefig(image_path, dpi = 400)
        plt.close()
        

    @property
    def model_dir(self):
        return "{}_{}_{}_{}".format(
            self.dataset_name, self.batch_size,
            self.output_height, self.output_width)

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name+'.model'), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['D_losses'])+1)

        y1 = hist['D_losses']
        y2 = hist['G_losses']

        plt.plot(x, y1, label='D_loss')
        plt.plot(x, y2, label='G_loss')

        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        plt.legend(loc=4)
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
        
    def visualize_epoch_results(self, epoch):
        tot_num_samples = min(self.sample_num, self.batch_size) # 100
        image_frame_dim = int(np.floor(np.sqrt(tot_num_samples))) # 10
        
        """ random condition, random noise """
            
        noise = np.random.normal(0, 1, (self.visual_team, 1, 1, self.z_dim))
        fixed_noise = noise 
        fixed_label = np.zeros((self.visual_team , 1))
        
        #用于显示25组图像
        for i in range(self.nb_class - 1):
            fixed_noise = np.concatenate([fixed_noise, noise], 0)
            temp_label = np.ones((self.visual_team, 1)) + i
            fixed_label = np.concatenate([fixed_label, temp_label], 0)
        fixed_label = self.onehot[fixed_label.astype(np.int32)].reshape((self.batch_size, 1, 1, self.nb_class))
    
        samples = self.sess.run(self.fake_images, feed_dict={self.z: fixed_noise, self.y_label: fixed_label})

        self.save_matplot_img(samples[:image_frame_dim * image_frame_dim, :, :, :], [image_frame_dim, image_frame_dim],
                    self.result_dir + '/' + self.model_name + '_epoch%03d' % epoch + '_' + self.dataset_name +'.png')

   

In [6]:
gan_type = 'cDCGAN'
dataset = 'Class2_Sample1_test1_1_1000'
epoch = 200
batch_size = 100
checkpoint_dir = 'checkpoint'
result_dir = 'results'
log_dir = 'logs'
test_dir = 'image_gan'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --result_dir
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# --result_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --test_dir
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    cDCGAN = cDCGAN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                    result_dir=result_dir, log_dir=log_dir, test_dir=test_dir, trainhist_dir=trainhist_dir)

    # build graph
    cDCGAN.build_model()

    # show network architecture
    cDCGAN.show_all_variables()

    # launch the graph in a session
    cDCGAN.train()
    
    #visualize_img_to_gif
    cDCGAN.visualize_img_to_gif(path=result_dir, size=(900,600), fps=5)

    vepoch = random.sample(range(100,170), 20)

    for e in tqdm(vepoch):
        cDCGAN.visualize_test_image( e, 100, 1)
        
sess.close()
        

D:x (100, 128, 128, 3)
D:y_fill (100, 128, 128, 2)
D: (100, 64, 64, 32)
D: (100, 64, 64, 32)
D: (100, 64, 64, 64)
D: (100, 32, 32, 64)
D: (100, 16, 16, 128)
D: (100, 8, 8, 256)
D: (100, 4, 4, 512)
D:out_logit (100, 1, 1, 1)
D:out (100, 1, 1, 1)
------------------------
G:z (100, 1, 1, 100)
G:y_label (100, 1, 1, 2)
G: (100, 4, 4, 256)
G: (100, 4, 4, 256)
G: (100, 4, 4, 512)
G: (100, 8, 8, 512)
G: (100, 16, 16, 128)
G: (100, 32, 32, 64)
G: (100, 64, 64, 32)
G: (100, 128, 128, 3)
------------------------
D:x (100, 128, 128, 3)
D:y_fill (100, 128, 128, 2)
D: (100, 64, 64, 32)
D: (100, 64, 64, 32)
D: (100, 64, 64, 64)
D: (100, 32, 32, 64)
D: (100, 16, 16, 128)
D: (100, 8, 8, 256)
D: (100, 4, 4, 512)
D:out_logit (100, 1, 1, 1)
D:out (100, 1, 1, 1)
------------------------
G:z (100, 1, 1, 100)
G:y_label (100, 1, 1, 2)
G: (100, 4, 4, 256)
G: (100, 4, 4, 256)
G: (100, 4, 4, 512)
G: (100, 8, 8, 512)
G: (100, 16, 16, 128)
G: (100, 32, 32, 64)
G: (100, 64, 64, 32)
G: (100, 128, 128, 3)
-----------

Epoch: [193] [   3/  20] time: 53.2455, d_loss: 0.15125179, g_loss: 4.40341663
Epoch: [193] [   4/  20] time: 53.7190, d_loss: 0.13831598, g_loss: 3.21561694
Epoch: [193] [   5/  20] time: 54.2174, d_loss: 0.07806432, g_loss: 3.42737150
Epoch: [193] [   6/  20] time: 54.6966, d_loss: 0.11525726, g_loss: 4.02499151
Epoch: [193] [   7/  20] time: 55.1835, d_loss: 0.07602951, g_loss: 4.03896189
Epoch: [193] [   8/  20] time: 55.6650, d_loss: 0.08597717, g_loss: 3.88558483
Epoch: [193] [   9/  20] time: 56.1552, d_loss: 0.10427490, g_loss: 3.61598706
Epoch: [193] [  10/  20] time: 56.6566, d_loss: 0.11826041, g_loss: 3.66373324
Epoch: [193] [  11/  20] time: 57.1524, d_loss: 0.10792067, g_loss: 3.75782728
Epoch: [193] [  12/  20] time: 57.6174, d_loss: 0.11621810, g_loss: 3.16845083
Epoch: [193] [  13/  20] time: 58.1189, d_loss: 0.11068827, g_loss: 4.83276463
Epoch: [193] [  14/  20] time: 58.6083, d_loss: 0.23637965, g_loss: 1.80512583
Epoch: [193] [  15/  20] time: 59.0812, d_loss: 0.31

Epoch: [198] [   1/  20] time: 178.5441, d_loss: 0.13633618, g_loss: 3.43660259
Epoch: [198] [   2/  20] time: 179.0719, d_loss: 0.10035962, g_loss: 3.19329309
Epoch: [198] [   3/  20] time: 179.5885, d_loss: 0.10620913, g_loss: 3.69285059
Epoch: [198] [   4/  20] time: 180.0922, d_loss: 0.09932896, g_loss: 3.73386598
Epoch: [198] [   5/  20] time: 180.5906, d_loss: 0.14573434, g_loss: 3.22782421
Epoch: [198] [   6/  20] time: 181.0737, d_loss: 0.19139746, g_loss: 3.53449225
Epoch: [198] [   7/  20] time: 181.5626, d_loss: 0.08474525, g_loss: 4.28076172
Epoch: [198] [   8/  20] time: 182.0415, d_loss: 0.10746295, g_loss: 3.74169850
Epoch: [198] [   9/  20] time: 182.5191, d_loss: 0.10269394, g_loss: 3.16688800
Epoch: [198] [  10/  20] time: 183.0323, d_loss: 0.11954165, g_loss: 3.37635803
Epoch: [198] [  11/  20] time: 183.5232, d_loss: 0.09452444, g_loss: 4.20082808
Epoch: [198] [  12/  20] time: 184.0172, d_loss: 0.11220875, g_loss: 3.12910461
Epoch: [198] [  13/  20] time: 184.5111,

  1%|          | 1/198 [00:00<00:21,  8.98it/s]

Avg per epoch ptime: 9.81, total 200 epochs ptime: 253.17
 [*] Training finished!
 [*] Generation Animation GIF start!
 [*] Reading 198 (900, 600) images start!


...

 [*] Finished generating Epoch: 117 , 100 new images!
INFO:tensorflow:Restoring parameters from checkpoint/Class2_Sample1_test1_1_1000_100_128_128/cDCGAN/cDCGAN.model-169


 95%|█████████▌| 19/20 [01:39<00:05,  5.23s/it]

 [*] Finished generating Epoch: 169 , 100 new images!
INFO:tensorflow:Restoring parameters from checkpoint/Class2_Sample1_test1_1_1000_100_128_128/cDCGAN/cDCGAN.model-123


100%|██████████| 20/20 [01:45<00:00,  5.27s/it]

 [*] Finished generating Epoch: 123 , 100 new images!
